In [1]:
from transformers import AutoModelForSeq2SeqLM
import peft
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, IA3Config, TaskType
import torch
from datasets import load_dataset
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cuda"
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

checkpoint_name = "financial_sentiment_analysis_ia3_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 8e-3
num_epochs = 3
batch_size = 8

2023-08-07 13:30:23.456009: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import importlib

importlib.reload(peft)

<module 'peft' from '/nfs/hpc/share/balasuri/peft/src/peft/__init__.py'>

In [3]:
# creating model
peft_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, feedforward_modules=[])

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

Some weights of UMT5ForConditionalGeneration were not initialized from the model checkpoint at bigscience/mt0-large and are newly initialized: ['decoder.block.3.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.21.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.20.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.4.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.14.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.17.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.12.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.7.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.19.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.23.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.12.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.5.layer.0.SelfAttention.relative_attentio

In [4]:
model

UMT5ForConditionalGeneration(
  (shared): Embedding(250112, 1024)
  (encoder): UMT5Stack(
    (embed_tokens): Embedding(250112, 1024)
    (block): ModuleList(
      (0-23): 24 x UMT5Block(
        (layer): ModuleList(
          (0): UMT5LayerSelfAttention(
            (SelfAttention): UMT5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): UMT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): UMT5LayerFF(
            (DenseReluDense): UMT5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features

In [5]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

trainable params: 282,624 || all params: 1,229,887,488 || trainable%: 0.022979662998246553


PeftModelForSeq2SeqLM(
  (base_model): IA3Model(
    (model): UMT5ForConditionalGeneration(
      (shared): Embedding(250112, 1024)
      (encoder): UMT5Stack(
        (embed_tokens): Embedding(250112, 1024)
        (block): ModuleList(
          (0-23): 24 x UMT5Block(
            (layer): ModuleList(
              (0): UMT5LayerSelfAttention(
                (SelfAttention): UMT5Attention(
                  (q): Linear(in_features=1024, out_features=1024, bias=False)
                  (k): Linear(
                    in_features=1024, out_features=1024, bias=False
                    (ia3_l): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 1024x1])
                  )
                  (v): Linear(
                    in_features=1024, out_features=1024, bias=False
                    (ia3_l): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 1024x1])
                  )
                  (o): Linear(in_features=1024, out_features

In [6]:
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /nfs/stak/users/balasuri/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'UPM stock fell 3 percent to EURO 12.49 $ 17.24 in early afternoon trading in Helsinki .',
 'label': 0,
 'text_label': 'negative'}

In [7]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [8]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [9]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 12.54it/s]


epoch=0: train_ppl=tensor(1.7078, device='cuda:0') train_epoch_loss=tensor(0.5352, device='cuda:0') eval_ppl=tensor(1.1951, device='cuda:0') eval_epoch_loss=tensor(0.1783, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 12.48it/s]


epoch=1: train_ppl=tensor(1.2120, device='cuda:0') train_epoch_loss=tensor(0.1923, device='cuda:0') eval_ppl=tensor(1.1214, device='cuda:0') eval_epoch_loss=tensor(0.1146, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 12.46it/s]

epoch=2: train_ppl=tensor(1.1636, device='cuda:0') train_epoch_loss=tensor(0.1515, device='cuda:0') eval_ppl=tensor(1.1057, device='cuda:0') eval_epoch_loss=tensor(0.1004, device='cuda:0')


In [10]:
# print accuracy
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=91.62995594713657 % on the evaluation dataset
eval_preds[:10]=['negative', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral']
dataset['validation']['text_label'][:10]=['negative', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'positive']


In [11]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [12]:
ckpt = f"{peft_model_id}/adapter_model.bin"
!du -h $ckpt

1.5M	bigscience/mt0-large_IA3_SEQ_2_SEQ_LM/adapter_model.bin


In [13]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

Some weights of UMT5ForConditionalGeneration were not initialized from the model checkpoint at bigscience/mt0-large and are newly initialized: ['decoder.block.3.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.21.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.20.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.4.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.14.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.17.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.12.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.7.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.19.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.23.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.12.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.5.layer.0.SelfAttention.relative_attentio

In [14]:
model.eval()
i = 13
inputs = tokenizer(dataset["validation"][text_column][i], return_tensors="pt")
print(dataset["validation"][text_column][i])
print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

The group 's 12-month operating profit grew 31 percent to 337.8 million euros .
{'input_ids': tensor([[   486,   7666,    259,    277,    263,  33789,  30866,    259,  45814,
           8170,   6998,    481,   1606,  17937,    288,   3549, 112575,   8381,
          13666,    259,    260,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[    0, 18205,     1]])
['positive']
